In [1]:
import logging
import os
import random
import sys
from collections import deque
from operator import itemgetter

import gym_donkeycar
import gymnasium as gym
import imageio
import ipywidgets as widgets
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from IPython.display import display
from ipywidgets import HBox, VBox
from matplotlib import pyplot as plt
from PIL import Image
from ruamel.yaml import YAML
from scipy.ndimage import gaussian_filter1d
from scipy.stats import norm
from tensorboard import notebook
from tensorboard.backend.event_processing.event_accumulator import \
    EventAccumulator
from torch import distributions as dist
from torch.distributions import Categorical, Normal
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from tqdm import tqdm

# suppress warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["IMAGEIO_IGNORE_WARNINGS"] = "True"

import gym.spaces as gym_spaces
import gymnasium as gym  # overwrite OpenAI gym
import stable_baselines3 as sb3
from gym_donkeycar.envs.donkey_env import DonkeyEnv
from gymnasium import spaces
from gymnasium.spaces import Box
from stable_baselines3 import A2C, PPO, SAC
from stable_baselines3.common import env_checker
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

from src.actor_critic import ContinuousActorCritic
from src.blocks import CategoricalStraightThrough, ConvBlock
from src.categorical_vae import CategoricalVAE
from src.imagination_env import ImaginationEnv
from src.mlp import MLP
from src.preprocessing import grayscale_transform as transform
from src.replay_buffer import ReplayBuffer
from src.rssm import RSSM
from src.utils import (load_config, make_env, save_image_and_reconstruction,
                       to_np)
from src.vae import VAE

torch.cuda.empty_cache()
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load the config
config = load_config()
for key in config:
    locals()[key] = config[key]

## Init the RSSM (including all networks)

In [ ]:
rssm = RSSM().to(device)

## Create the imagination environment for training the agent

In [ ]:
replay_buffer = ReplayBuffer()

In [ ]:
imagination_env = ImaginationEnv(
    rssm,
    replay_buffer,
    render_mode=None
)

## Init the agent

In [ ]:
agent = PPO(
    policy="MlpPolicy",
    env=imagination_env,
    verbose=verbose,
    tensorboard_log=log_dir,
    gamma=gamma,
    gae_lambda=lam,
    ent_coef=ent_coef,
)

## Training loop


In [ ]:
""" training loop """

rssm.load_weights("weights/RSSM_1.70111713")
rssm.train()

# Create the environment
env = make_env()

# Logging
writer = SummaryWriter(log_dir)
if config["show_inline_tensorboard"]:
    notebook.start(f"--logdir={log_dir}")

episode_return = []
episode_losses = { # for loss plots
    "episode_loss": [],
    "episode_image_loss": [],
    "episode_reward_loss": [],
    "episode_continue_loss": [],
    "episode_dyn_loss": [],
    "episode_rep_loss": [],
}

best_running_loss = np.inf # used for saving the best rssm model

try:
    for episode in tqdm(range(start_episode, n_seed_episodes + n_training_episodes)):

        if verbose:
            print("EPISODE:", episode)
        
        # Get the initial state
        obs, info = env.reset()

        # Reset the RNN's hidden state
        h = torch.zeros(rssm.num_rnn_layers, 1, H, device=device) # seq_len, B, H

        # Add a new loss for the current episode and initialize it to 0
        episode_length = 0
        episode_return.append(0)
        for key in episode_losses:
            episode_losses[key].append(torch.tensor(0, device=device, dtype=torch.float32))

        # Play one episode
        done = False
        while not done:

            # preprocess the observation and add it to the replay buffer
            x = transform(obs).view(-1, 1 if grayscale else 3, *size) # (B, 3, 128, 128)
            replay_buffer.push(x)

            """ WORLD MODEL LEARNING """

            # predict z and generate the true stochastic latent variable z with the encoder
            z_prior = rssm.dynamics_mlp(h).view(-1, num_categoricals, num_classes) # (1,32,32) for the softmax
            z_prior = F.softmax(z_prior, -1).flatten(start_dim=1, end_dim=2) # (1, 1024)
            z = rssm.vae.encode(x).flatten(start_dim=1, end_dim=2)
            state = to_np(torch.cat((h.flatten().detach(), z.flatten().detach()), dim=0))
            
            # get random action for the first seed episodes
            # and in training mode get the action from the actor
            if episode < n_seed_episodes:
                action = env.action_space.sample()
            else:
                action, _ = agent.predict(state, deterministic=False)

            # predict one step using the RSSM
            h, reward_pred, continue_prob, continue_pred, x_pred = rssm.step(action, h, z)

            # take an environment step with the action
            obs, reward, terminated, truncated, info = env.step(action.squeeze())
            done = terminated or truncated
            
            # Add the reward to the episode return
            episode_return[-1] += reward

            # calculate the loss
            continue_target = torch.tensor(1 - done, device=device, dtype=torch.float32)
            reward = torch.tensor(reward, device=device, dtype=torch.float32)
            losses = rssm.get_losses(x, x_pred, reward, reward_pred, 
                                     continue_target, continue_prob, z_prior, z)

            # Add the loss for the current step to the episode loss
            episode_length += 1
            for key in losses:
                episode_losses["episode_" + key][-1] += losses[key]

        # Calculate the mean loss of the episode
        for key in episode_losses:
            episode_losses[key][-1] /= episode_length

        # update the world model at the end of an episode using the mean loss of the episode
        rssm.optim.zero_grad()
        episode_losses["episode_loss"][-1].backward()
        nn.utils.clip_grad_norm_(rssm.vae.parameters(), max_norm=max_grad_norm, norm_type=2)  
        rssm.optim.step()

        # Log the episode return
        writer.add_scalar("episode_return", episode_return[-1], global_step=episode)
        
        # Detach the losses to save memory and log them in TensorBoard
        for key in episode_losses:
            episode_losses[key][-1] = episode_losses[key][-1].detach().item()
            writer.add_scalar(key, episode_losses[key][-1], global_step=episode)
        
        # save original image and reconstruction
        if episode % 10 == 0:
            save_image_and_reconstruction(x, x_pred, episode)
        
        # save the rssm model with the best running loss
        running_loss = np.mean(episode_losses["episode_loss"][-10:])
        if episode > 0 and episode % 10 == 0 and running_loss < best_running_loss:
            best_running_loss = running_loss
            
            # save the rssm and agent
            rssm.save_weights(filename=f"RSSM_{best_running_loss:.8f}")
            agent.save(f"weights/{agent.__class__.__name__}_agent")
        
        """ RL AGENT LEARNING (IN THE WORLD MODEL) """
        if episode >= n_seed_episodes:
            if verbose:
                print("AGENT IS LEARNING")
            agent.learn(
                total_timesteps=imagination_timesteps_per_model_update,
                progress_bar=imagination_progress_bar,
                reset_num_timesteps=False
            )

    env.close()

except KeyboardInterrupt:
    """ Clean handling for interrupts to stop training early """
    print("Stopping training.")
    # Delete the last loss if the training was stopped early
    # so that the list only consists of floats
    for key in episode_losses:
        if isinstance(episode_losses[key][-1], torch.Tensor):
            episode_losses[key] = episode_losses[key][:-1]

    # Close the TensorBoard writer and the gym environment
    writer.close()
    env.close()

## Plot the results

In [ ]:
rolling_length = max(1, int(len(episode_losses["episode_loss"])/20))

fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(3*5, 2*5))

# Iterate over the keys and plot the losses
for i, key in enumerate(episode_losses.keys()):
    row = i // 3
    col = i % 3

    axs[row, col].set_title(key)
    losses = episode_losses[key]
    losses_moving_average = (
        np.convolve(
            np.array(losses).flatten(), np.ones(rolling_length), mode="valid"
        )
        / rolling_length
    )
    axs[row, col].plot(range(len(losses)), losses, label=key)
    axs[row, col].plot(range(len(losses_moving_average)), losses_moving_average, label="moving average")
    axs[row, col].legend(loc="upper right")

plt.tight_layout()
plt.show()

## Showcase the trained agent playing in latent imagination

In [ ]:
showcase_agent = True

if showcase_agent:
    
    showcase_rewards = []
    imagination_env.render_mode = "gif"
    obs, info = imagination_env.reset()
    
    for i in range(500):
        
        # apply the RL agent in eval mode to get an action
        state = to_np(torch.cat((h.flatten().detach(), z.flatten().detach()), dim=0))
        action, _ = agent.predict(state, deterministic=False)
        # action = imagination_env.action_space.sample()
        
        obs, reward, terminated, truncated, info = imagination_env.step(action)
        showcase_rewards.append(reward)
        imagination_env.render()
        
        if terminated or truncated:
            break
        
    imagination_env.close()
    imagination_env.render_mode = None

plt.plot(showcase_rewards)

## Test area

In [ ]:
# !tensorboard 